# Homework 4

## 2. Recurrent Network (NN) Classification

In [21]:
import os
import numpy as np
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [22]:
!unzip "UCI HAR Dataset.zip"
!unzip "/content/human+activity+recognition+using+smartphones.zip"

Archive:  UCI HAR Dataset.zip
replace UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/UCI HAR Dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace UCI HAR Dataset/activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/UCI HAR Dataset/._activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace UCI HAR Dataset/features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/UCI HAR Dataset/._features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/UCI HAR Dataset/._features.txt  
  inflating: UCI HAR Dataset/features_info.txt  
  inflating: __MACOSX/UCI HAR Dataset/._features_info.txt  
  inflating: UCI HAR Dataset/README.txt  
  inflating: __MACOSX/UCI HAR Dataset/._README.txt  
  inflating: UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt  
  inflating: __MACOSX/UCI HAR Dataset/test/Inertial Signals/._body_acc_x_test.txt  
  inflating: UCI HAR Dataset/test/Inertial Si

In [23]:
# Helper function to load the raw inertial signals
def load_ucihar(data_dir="UCI HAR Dataset", subset="train"):
    """ Loads the UCI HAR data from the Inertial Signals folder. Returns: X: numpy array of shape (num_samples, seq_len, num_signals) y: numpy array of labels (0-indexed) """
    # The nine signal types available in the dataset
    signal_types = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z" ]
    signals = []
    # Each signal file is located in {data_dir}/{subset}/Inertial Signals/
    for signal in signal_types:
        filename = os.path.join(data_dir, subset, "Inertial Signals", f"{signal}_{ subset}.txt")
        # Each file has shape (num_samples, 128)
        data = np.loadtxt(filename) # Add a new axis so that we can later stack to shape (num_samples, 128, num_signals)
        signals.append(data[..., np.newaxis]) # Stack along the last dimension to form (num_samples, 128, 9)
    X = np.concatenate(signals, axis=2) # Load labels from y_{subset}.txt; labels in the dataset are 1-indexed, so subtract 1.
    y_path = os.path.join(data_dir, subset, f"y_{subset}.txt")
    y = np.loadtxt(y_path).astype(int)- 1
    return X, y

In [24]:
# Define a PyTorch Dataset for UCI HAR
class UCIHARDataset(Dataset):
  def __init__(self, data_dir="UCI HAR Dataset", subset="train"):
    self.X, self.y = load_ucihar(data_dir, subset)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    sample = torch.tensor(self.X[idx], dtype=torch.float32)
    label = torch.tensor(self.y[idx], dtype=torch.long)
    return sample, label

In [25]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

default_input_dim = 9
default_feature_dim = 16
default_num_classes = 6

# define model
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x. device)
        out, _ = self.rnn(x, h0)
        logit = self.fc(out[:,-1, :])
        prob = nn.functional.softmax(logit, dim=1)
        return prob, logit


In [26]:
num_epochs, batch_size, lr = 10, 16, 0.001

# create train, test dataset
train_dataset = UCIHARDataset(subset="train")
test_dataset = UCIHARDataset(subset="test")

#    create train, test loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle= False)

# create model, loss criterion, optimizer
model = RNNClassifier(input_size=9, hidden_size=16, num_layers=1, num_classes=6)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [27]:
def train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs):
    # start training
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            activity_prob, activity_logit = model(inputs)
            loss = criterion(activity_logit, labels)
            loss.backward()
            optimizer.step()

    # Evaluate on test data
    model.eval()
    test_acc = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            activity_prob, activity_logit = model(inputs)
            _, predicted = torch.max(activity_logit.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = correct / total
    print(f"Test Accuracy: {test_acc:.4f}")

In [28]:
# Evaluate on test data
train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)

Test Accuracy: 0.7000


In [29]:
feature_dim = 64

# create model, loss criterion, optimizer
model = RNNClassifier(input_size=9, hidden_size=feature_dim, num_layers=1, num_classes=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)

Test Accuracy: 0.6064


In [30]:
feature_dim = 16
num_layers = 2

# create model, loss criterion, optimizer
model = RNNClassifier(input_size=9, hidden_size=feature_dim, num_layers=num_layers, num_classes=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)

Test Accuracy: 0.6532


In [ ]:
num_layers = 3

# create model, loss criterion, optimizer
model = RNNClassifier(input_size=9, hidden_size=16, num_layers=num_layers, num_classes=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)

In [ ]:
num_layers = 4

# create model, loss criterion, optimizer
model = RNNClassifier(input_size=9, hidden_size=16, num_layers=num_layers, num_classes=6)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)

In [ ]:
# Replace RNN with LSTM (feature_dim=16, num_layers=1)
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        logit = self.fc(out[:, -1, :])
        prob = nn.functional.softmax(logit, dim=1)
        return prob, logit

feature_dim = 16
num_layers = 1
input_size = 9
num_classes = 6

# create model, loss criterion, optimizer
model = LSTMClassifier(input_size=input_size, hidden_size=feature_dim, num_layers=num_layers, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_model_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs)